[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/redcican/Coursera_Capstone/blob/master/Second_Assignment_Neighborhood_Segmentation_and_Clustering.ipynb)

# IBM Coursera Final Project 1: Segmenting and Clustering Neighborhoods in Toronto - Second Assignment

## 1. Import the Libraries

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [0]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).text

Use Beautifulsoup to extract the table content from URL page.

In [0]:
soup = BeautifulSoup(html, 'html.parser')

Find the table, and check the content.

In [0]:
my_table = soup.find('table',{'class':'wikitable sortable'})
my_table

Only find all cells.

In [0]:
cells = my_table.find_all('td')
cells

Define 3 lists for 3 columns.

In [0]:
Postcode = [];
Borough = [];
Neighbourhood = [];

A for loop to iterate all data.

In [0]:
for row in my_table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        Postcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True))

Convert the lists to a Pandas Dataframe, and check the first 10 rows.

In [58]:
df = pd.DataFrame({"Postcode":Postcode,
                  "Borough":Borough,
                  "Neighbourhood":Neighbourhood})
df = df[['Postcode','Borough','Neighbourhood']]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Ignore cells with a borough that is **Not assigned**.

In [59]:
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Using Pandas `groupby`function to combine two rows into one row with the neighborhoods separated with a comma.

In [60]:
df['Neighbourhood'] = df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ', '.join(x))
df = df.drop_duplicates()
df = df.replace(r'\n','', regex=True) 
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough.

In [0]:
for i in range(len(df['Neighbourhood'])):
    if df['Neighbourhood'].iloc[i] == 'Not assigned':
        df['Neighbourhood'].iloc[i] = df['Borough'].iloc[i]

In [62]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


Use the **.shape** method to print the number of rows.

In [63]:
df.shape

(103, 3)

# Second Assignment

In [14]:
!pip install geocoder
import geocoder

    100% |████████████████████████████████| 102kB 4.3MB/s 
    100% |████████████████████████████████| 81kB 7.4MB/s 


In [0]:
'''
latitude = []
longitude = []

for code in Postcode:

    lat_lng_coords = None

    # loop until  get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print(code, 'is done!')
'''

Because the geographical coordinates of the neighborhoods using the Geocoder package are not available, I use provided CSV file to merge the final DataFrame.

In [18]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates.csv
User uploaded file "Geospatial_Coordinates.csv" with length 2891 bytes


In [64]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.rename({'Postal Code':'Postcode'},axis=1,inplace=True)
df_geo.head(10)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Use `merge` function to combine two csv files.

In [70]:
df = df.merge(df_geo,left_on='Postcode',right_on='Postcode')
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
